In [1]:
from src.gmail import Gmail
from src.newsletters.config import NEWSLETTER_AND_PARSER
from src.newsletters.parser.tldr import tldr_parser
from src.newsletters.parser.alpha_signal import alpha_signal_parser
from src.newsletters.parser.bloomberg_tech import bloomberg_tech_parser
from src.newsletters.parser.tech_crunch import tech_crunch_parser
from src.config import SCOPES
from src.newsletters.parser.process_links import process_raw_url

import re

In [2]:
import logging
logging.basicConfig(
    level=logging.ERROR,
    format="%(asctime)s - %(name)s - %(levelname)s -- l.%(lineno)d: %(message)s",
)

In [3]:
NEWSLETTER_AND_PARSER.keys()

dict_keys(['TLDR AI <dan@tldrnewsletter.com>', 'AlphaSignal <news@alphasignal.ai>', 'TLDR <dan@tldrnewsletter.com>', 'TLDR Product <dan@tldrnewsletter.com>', 'Bloomberg Technology <noreply@news.bloomberg.com>', 'TechCrunch <newsletters@techcrunch.com>'])

In [4]:
gmail = Gmail(
    path_to_token="/home/secrets_vault/token.json",
    path_to_credentials="/home/secrets_vault/credentials.json",
    scopes=SCOPES
)

# TechCrunch

In [6]:
emails = gmail.fetch_emails(sender='TechCrunch <newsletters@techcrunch.com>', after='2025-02-07', before='2025-02-07')
len(emails)

1

In [7]:
news_sources = tech_crunch_parser(emails[0])

In [8]:
news_sources

[{'title': 'Joining forces',
  'url': 'https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VWk1PJ7_QZRGW2G7P1p8PzjY1W7FnbcL5rMzw3N7Rflzb3gP0xW95jsWP6lZ3ljN2w796N2Gl1vVT5_N86hKmMzW5ZWwCk89FtJfW2q9sGS1RRlzVW3LWWXb37n8t9W4VsSrD4lk4w6W2jPfh85sD2t8W5_nWgv50fNb2V7z9ms6rnxtCW8srTmc4QpWXMW4T6h7t3dtnwMW6Z89TC7rVqprW58CVcH7Y288FW51PVXd5zYJQ0V48cZF5dFSsgVxzWjM8655k8W7CvW0b80s7GJW7nrZCH7tLMxyW5gjnzT3kgwbvW7Jbfr12HsnsXW7x7t7G2N8hpGW1XGq-j8pRXT7W2LCG8V52S0y6W6XMc_12lclbqW1-d_Bf47vhqDW8Q6hnh2Kx73fW582wP_52LL7BW89RPqn50dT5kMfhb-bGBv8nW7phv2061mWwGf1pDK_Y04 ',
  'news_provider': 'techcrunch.com',
  'source_of_the_news': 'TechCrunch <newsletters@techcrunch.com>',
  'text': '',
  'news_summary': 'Following a similar partnership with Aurora Innovation, Volvo Autonomous Solutions partnered with self-driving truck startup Waabi  to jointly develop and deploy autonomous trucks. Commercial pilots are set to launch in Texas soon, with a driverless demonstration on public roads planned for the end of the year.

In [21]:
re.sub(r"\(https://[^\)]+\)", "", news_sources[0].split("\r\n\r\n")[-3])

'Security M&As: Security compliance firm Data acquired software security review startup SafeBase for $250 million  . And CyberArk, an identity security company, bought startup Zilla Security  for $165 million in cash, with an additional $10 million contingent on certain milestones.'

In [9]:
MARKERS = [
    "Most interesting startup stories from the week",
    "Most interesting VC and funding news this week",
    "Last but not least",
]

In [14]:
mark = MARKERS[0]
txt.split(mark)[1].split("\r\n\r\n")

['',
 'Arthur Mensch of Mistral AI (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNCl3gP0xW7Y8-PT6lZ3lJW47D_Kx3yMYscW4LYgwP7CYWkqV4tzqW3FZ0QbW1k8FrF8dgqm_W48SMsr1T--0nN990vdQljlc6N5lTtBT1xSK0W7LkYJR1RZKMwW1lbYLw7617WFVBbyfs26zyb8VZn6sh5tVlvGW5LF4M98k_jT0W25f2zq9kzYFfW42HYd46R3RlDW1_kX5K3wQlLSMmtYZKZnjX3W33ScTb6KCGVwW4jS5HT87C4rsVyjC7w5WvtxfN30Sk9l1CWxsW3tbc6w4dbHrmW1c2GgN2kqyvhW5SXSfY6755v-W5t0kX46PqQg9W4Vngpp5k-4_ZW7F7rpJ2NBFRmf1GN_P804 )',
 'Image Credits:  Nathan Laine / Bloomberg / Getty Images',
 'Uncertainty usually doesn’t rhyme with investment. But there’s always money for security, whether online or IRL.',
 'Countercyclical: AI startups raised $110 billion in 2024 (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNB85fkwKW5BWr2F6lZ3n6V1c0Gt6nz-6fW9j8xhp7Fhk5QW3WsY_b5MR4nDW60BhXD328ZgGN5dhMc7vnhvKV3WDZ_4dt8--W7jzQYn7KKQPWW6jH3w08MxqGzW1Ct3rH3TmwfgW83XG4q6ZsZs6W1BMTWM4Y0PfLN7pHDF-sygf6W8

In [45]:
# Isolate ads with '\r\n\r\n\u200a\r\n\r\n\u200a\r\n\r\n'
split_ads = txt.split("\r\n\r\n\u200a\r\n\r\n\u200a\r\n\r\n")
print(len(split_ads))
for tt in split_ads:
    print(tt)
    print("\n\n" + "="*60 + "\n\n")

3
Plus, Databricks closes a $10B Series J

View this email online in your browser (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VWsW7P5D_y4YW68y4sS7Zv_0CW5l3Nw35rbzG7MGxfTK5m_5PW6N1X8z6lZ3pbW69k03S14RPCnW3NZNgP2QBBzjW2YF2Qg5Tg6MjVhWY6G3y6lmkW1CXVNn349FwdVz3trg4SGCcwW8qrVJg8klW3RW2gRhkn7N4v5TW5t05ST703wqpW19Yjnw1Q0XNGW8vd2ph8DsjfPN8_rcpvlWjkBW7r_Cth7_DlShW3nhgD36c4nlsN707lwJMBgVbW2fGCZ_2KzZ-PW5f8qjd4TBHzrW1vWXt47KlF3GW7ZRp4f9fx-XkW2Rm8l32XjbVnW3C6Drd7SrnCrW5fWgDB63BxypW8yHQjX66-BBYW6q2Gmz2NbPHbW1kFPjy81nBVXW5mbTqb70XM6mW5-KnZl8pWq94W952Sc836Ryq3W95tB031-KKvsW4jGv7v2JpZycN7sRCpzmG030N3g3LRQs91wtW6vX5Nd3v0w_3W2gX_TQ89dbfRW8d40FQ7wwz5TW8Z55Yj92M2f-W21P8W25M5G2kW4crQlz2qb0xlf1Fj5Dq04 )

By Anna Heim (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VWsW7P5D_y4YW68y4sS7Zv_0CW5l3Nw35rbzG7MGxfTq3gP0xW69sMD-6lZ3p1N7zXgdL67vW9W43wPvp1cLHCtW2tncpw6nx_-NW91xpPL9lwZZVW31qdBK1Tq0swW3S2m-b6nDRmBN1cHS67kLn7MW6q6s7b1235T4VP11Rt5stznFW7CQnHR8QjkkRW60QPmB4YR0XpW8NL73m6w91HMN1XMkSK1JLw8VzY7rB4

In [16]:
txt_split = txt.split(MARKERS[0])[1].split(MARKERS[1])[0].split("\r\n\r\n")

In [19]:
"https://" in txt_split[4]

True

In [25]:
txt_split[-3]

'Security M&As: Security compliance firm Data acquired software security review startup SafeBase for $250 million (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNCY3gP0xW95jsWP6lZ3pPW73q-vZ1GHW5bW6ltW6G4MjR-yW84sZF32m5fY5N6YlqMv2RRfkW8mNfh93lr5FTW80sX787HPTVGW79Y9WZ51PdpDW7TmJCF6L9JlQVWj6_H3r9tS-W7PpGj64vxsmtW6fPKDJ6sRB3RW23yfFD1jmzVrW66PrgY7V5DHcVMwJ3p86F_yZW8DmMJq78TnsjW3WxLCC2DBBZrW1QXHq97TMTf4W7PZGTd5xRWN2W3Z_xTc1w0fbyW8CwR171nsl0cN8RLs5lNwnpbW5sZztx5VM7hsW5mqrZN8vpwCMW7W4BGH37YZrlW78f3tx44NfG4W9cCqNT5DZ0SgW1Gk-Zs89Xj6fW20gNJT5LhhzlW9kD4bn7Z3nQcW1NwfSR2nxcM0f7qrWgH04 ) . And CyberArk, an identity security company, bought startup Zilla Security (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNCF3gP0xW8wLKSR6lZ3p2W5h785-8BH3MmW4S5fcG8gTvzJW3hc9H72mFZb7W2hpY2Z7pSLz_W871yZf5HnV5JVlcB9b4N_JnJW38HBdc94bQ68W7KFV5l3dRxqlW41fJm31vfKnwW5w9GDg4NNw55V86FpL7NC6s6W50S8lN685x90W3dhP7b9849Q7VMdJnp4bX5yNW

In [26]:
import re
re.findall(r"\(https://[^\)]+\)",txt_split[-3])

['(https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNCY3gP0xW95jsWP6lZ3pPW73q-vZ1GHW5bW6ltW6G4MjR-yW84sZF32m5fY5N6YlqMv2RRfkW8mNfh93lr5FTW80sX787HPTVGW79Y9WZ51PdpDW7TmJCF6L9JlQVWj6_H3r9tS-W7PpGj64vxsmtW6fPKDJ6sRB3RW23yfFD1jmzVrW66PrgY7V5DHcVMwJ3p86F_yZW8DmMJq78TnsjW3WxLCC2DBBZrW1QXHq97TMTf4W7PZGTd5xRWN2W3Z_xTc1w0fbyW8CwR171nsl0cN8RLs5lNwnpbW5sZztx5VM7hsW5mqrZN8vpwCMW7W4BGH37YZrlW78f3tx44NfG4W9cCqNT5DZ0SgW1Gk-Zs89Xj6fW20gNJT5LhhzlW9kD4bn7Z3nQcW1NwfSR2nxcM0f7qrWgH04 )',
 '(https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNCF3gP0xW8wLKSR6lZ3p2W5h785-8BH3MmW4S5fcG8gTvzJW3hc9H72mFZb7W2hpY2Z7pSLz_W871yZf5HnV5JVlcB9b4N_JnJW38HBdc94bQ68W7KFV5l3dRxqlW41fJm31vfKnwW5w9GDg4NNw55V86FpL7NC6s6W50S8lN685x90W3dhP7b9849Q7VMdJnp4bX5yNW68_D8W7SDHBxN8L5ZHLjKBgkW1TT1kK7bSLFfW55V0Rl486LF5N3T0f7BznChSW146Xqt56xsNVW5bgC-l2LftJbW5XqM9S4dcQQvW89dpnd6-K4y4W8fMTMK61yVfxN7jvJmZSFCMVW8YRWFH7b_vhSW81bpgC4MQVr0W3D6Z6x8dYF2Qf1d0Dr

In [27]:
txt_split[4]

'Countercyclical: AI startups raised $110 billion in 2024 (https://email.techcrunch.com/e3t/Ctc/ZX+113/d5b44W04/VW9vbz66q9DsN5m5xQ5Cldx-W11fj5c5s1Bg9N31NNB85fkwKW5BWr2F6lZ3n6V1c0Gt6nz-6fW9j8xhp7Fhk5QW3WsY_b5MR4nDW60BhXD328ZgGN5dhMc7vnhvKV3WDZ_4dt8--W7jzQYn7KKQPWW6jH3w08MxqGzW1Ct3rH3TmwfgW83XG4q6ZsZs6W1BMTWM4Y0PfLN7pHDF-sygf6W8dklpS1P0HGSN4PpWWn1L3tGW1B5jM_4D7NwzVsG79J3cR8PGW20JFcN4_lsJXW2163bP4H8zKqW7l90-d5cS1cYW8PdPcd3CZ320W2TdGFs3lkQmYW3cpNhN3HcKl8W4cVxMv2ntHMPW82VCQl5wTy4DN4vYrwh4l5TWW1gVD9h8xtG85W158r015Cgr-rW3LMHj77wJ0_-W5knf-B8cR7-4VS2-v02yf0ctMZdYMqz5HWjW331dFB6mslx1W4Q24Xf8x_dTrW239Ynq1ZrmRrf4RHZtT04 ) , a 62% increase compared to 2023, while overall startup funding was down 12% year on year.'

In [32]:
("").join(txt_split[4].split(re.findall(r"\(https://[^\)]+\)",txt_split[4])[0]))

'Countercyclical: AI startups raised $110 billion in 2024  , a 62% increase compared to 2023, while overall startup funding was down 12% year on year.'

# Bloomberg Technology

In [5]:
emails = gmail.fetch_emails(sender='Bloomberg Technology <noreply@news.bloomberg.com>', after='2025-03-05', before='2025-03-05')
len(emails)

1

In [6]:
news_sources = bloomberg_tech_parser(emails[0])

In [7]:
news_sources

[{'title': 'New iPads',
  'url': 'https://www.bloomberg.com/news/articles/2025-03-04/apple-launches-upgraded-ipad-air-in-bid-to-sustain-tablet-growth?cmpid=tech-in-brief',
  'news_provider': 'www.bloomberg.com',
  'source_of_the_news': 'Bloomberg Technology <noreply@news.bloomberg.com>',
  'text': '',
  'news_summary': 'Apple introduced a new iPad Air and an upgraded low-end tablet, trying to capitalize on therenewed popularity  of the products while sales of the iPhone slump.',
  'date_source': 'Wed, 05 Mar 2025 13:08:45 +0000',
  'date_source_time_zone': 'utc'},
 {'title': 'French satellites',
  'url': 'https://www.bloomberg.com/news/articles/2025-03-04/starlink-rival-eutelsat-surges-on-european-defense-spending-plan?cmpid=tech-in-brief',
  'news_provider': 'www.bloomberg.com',
  'source_of_the_news': 'Bloomberg Technology <noreply@news.bloomberg.com>',
  'text': '',
  'news_summary': 'Eutelsat Communications SA shares jumped this week . Investors bet the French satellite provider wi

## Experimentation

In [9]:
process_raw_url("https://www.bloomberg.com/newsletters/tech-in-depth/latest")

('https://www.bloomberg.com/newsletters/tech-in-depth/latest',
 'www.bloomberg.com')

In [11]:
import requests
response = requests.get("https://www.bloomberg.com/newsletters/tech-in-depth/latest", allow_redirects=True)

In [12]:
response.url

'https://www.bloomberg.com/newsletters/tech-in-depth/latest'

In [14]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 12.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 11.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [15]:
from selenium import webdriver

url = "https://www.bloomberg.com/newsletters/tech-in-depth/latest"

driver = webdriver.Chrome()  # Make sure you have ChromeDriver installed
driver.get(url)

print(driver.current_url)  # This should show the final URL
driver.quit()

WebDriverException: Message: Service /root/.cache/selenium/chromedriver/linux64/133.0.6943.141/chromedriver unexpectedly exited. Status code was: 127


In [14]:
messages = gmail.fetch_emails(sender='Bloomberg Technology <noreply@news.bloomberg.com>', after='2025-03-05', before='2025-03-05')

In [16]:
messages

[{'sender': 'Bloomberg Technology <noreply@news.bloomberg.com>',
  'subject': "Trump assails Biden's Chips Act",
  'date_utc': 'Wed, 05 Mar 2025 13:08:45 +0000',
  'id': '195666c5783c7f6e',
  'text': "Tech In Brief \n Hi, you’re receiving our new free Tech In Brief newsletter because you had \nbeen getting one of Bloomberg’s technology newsletters that are n \n\n\n <https://bloom.bg/41JkELf> \n \n<https://sli.bloomberg.com/click?s=1105649&stpe=default&li=12803590&m=412ded93419b0667798b58cd796b87cf&p=03052025&us_privacy=1YYY>\n\n\n\n\n Get exclusive scoops, insights and analysis from Bloomberg technology \nreporters with the Tech Newsletter Bundle subscription. Includes the \nsubscriber-only newsletters Tech In Depth, Power On, Q&AI, Game On and \nSoundbite and paywall-free access to the links in them. \n Subscribe now  <https://bloom.bg/3WjXMhT> \n\n\n\n\nHi, you’re receiving our new free Tech In Brief newsletter because you had \nbeen getting one of Bloomberg’s technology newsletters 

In [17]:
txt = messages[0]['text']

In [18]:
txt

"Tech In Brief \n Hi, you’re receiving our new free Tech In Brief newsletter because you had \nbeen getting one of Bloomberg’s technology newsletters that are n \n\n\n <https://bloom.bg/41JkELf> \n \n<https://sli.bloomberg.com/click?s=1105649&stpe=default&li=12803590&m=412ded93419b0667798b58cd796b87cf&p=03052025&us_privacy=1YYY>\n\n\n\n\n Get exclusive scoops, insights and analysis from Bloomberg technology \nreporters with the Tech Newsletter Bundle subscription. Includes the \nsubscriber-only newsletters Tech In Depth, Power On, Q&AI, Game On and \nSoundbite and paywall-free access to the links in them. \n Subscribe now  <https://bloom.bg/3WjXMhT> \n\n\n\n\nHi, you’re receiving our new free Tech In Brief newsletter because you had \nbeen getting one of Bloomberg’s technology newsletters that are now \nsubscriber-only. You can manage your newsletter subscriptionshere \n<https://bloom.bg/41nNwZ7>. \n\n\n\n Tech Across the Globe \n\n\n\nNew iPads: Apple introduced a new iPad Air and an 

In [19]:
_text_txt = "Revalued \n\n\n\nAescape, the maker of an AI-powered massage robot <https://bloom.bg/41FwyFs>, \nraised $83 million in a new funding round that valued it at $250 million. The \ncompany has 40 robot massage tables on the market, including some at Equinox \ngyms, and plans to ship 400 more this year.\nA woman lies face down on an Aescape table, receiving a massage. Source: \nSource: Aescape \n\n\n"

In [23]:
(" ").join(_text_txt.strip().split())

'Revalued Aescape, the maker of an AI-powered massage robot <https://bloom.bg/41FwyFs>, raised $83 million in a new funding round that valued it at $250 million. The company has 40 robot massage tables on the market, including some at Equinox gyms, and plans to ship 400 more this year. A woman lies face down on an Aescape table, receiving a massage. Source: Source: Aescape'

# ALPHA SIGNAL

In [12]:
emails = gmail.fetch_emails(sender='AlphaSignal <news@alphasignal.ai>', after='2025-02-10', before='2025-02-10')
len(emails), type(emails)

(1, list)

In [13]:
news_sources = alpha_signal_parser(emails[0])

In [10]:
news_sources

[{'title': 'OpenAI’s Sam Altman outlines AI trends',
  'url': 'https://blog.samaltman.com/three-observations',
  'source_of_the_news': 'AlphaSignal <news@alphasignal.ai>',
  'text': 'OpenAI CEO Sam Altman outlined key trends in AI development, emphasizing the scaling economics of intelligence, the rapid cost reduction of AI usage, and the implications of AI agents.  KEY HIGHLIGHTS   \t* Intelligence scales logarithmically with resources, with no clear saturation point.   \t* AI costs decline at an unprecedented rate, dropping 10x every 12 months.   \t* AI agents are positioned as scalable knowledge workers, handling constrained tasks at scale.   \t* Scientific progress is expected to accelerate, driven by automation of complex reasoning tasks.   \t* AI will integrate across economic sectors, reshaping workflows without immediate large-scale disruptions.  ECONOMIC IMPLICATIONS   \t* AI spreads beyond tech companies, distributing its benefits across industries.   \t* The balance between 

In [23]:
news_sources[0]['url']

'https://mistral.ai/en/news/all-new-le-chat'

In [32]:
import pandas as pd
df = pd.read_parquet("/home/logs/database_news_stories.parquet")

In [33]:
df.dtypes

title                                           object
url                                             object
news_provider                                   object
source_of_the_news                              object
text                                            object
news_summary                                    object
date_source                 datetime64[ns, US/Eastern]
date_source_time_zone                           object
version                                         object
competitive_intelligence                        object
themes                                          object
market_intelligence                             object
personalities                                   object
score_category_count                             int64
unique_id                                        int64
dtype: object

## Experimentation

In [6]:
emails[0].keys()

dict_keys(['sender', 'subject', 'date_utc', 'id', 'text'])

In [7]:
emails[0]['sender']

'AlphaSignal <news@alphasignal.ai>'

In [21]:
emails[0]['subject']

'🔥Mistral AI updates Chatbot, Faster than ChatGPT & Claude'

In [25]:
(emails[0]['text'])

"On Andrew Ng's agentic object detection, GitHub Copilot's agent mode,\r\nGoogle's Imagen 3 on Gemini API,...\xa0\r\n\r\nSIGNUP [1]\xa0\xa0|\xa0\xa0WORK WITH US [2]\xa0\xa0|\xa0\xa0FOLLOW ON X\r\n[3]\xa0\xa0|\xa0\xa0READ ON WEB [4]\r\n\r\n . \r\n\r\nHEY ,\r\n\r\nWelcome to AlphaSignal – the most read newsletter by AI\r\ndevelopers.\xa0\r\n\r\nWe bring you the top 1% of news, papers, models, and repos, all\r\nsummarized to keep you updated on the latest in AI.\r\n\r\nIN TODAY'S SIGNAL\r\n\r\n_Read time: 5 min 41 sec_\r\n\r\n🎖️ TOP NEWS\r\n\r\n \t*\r\nMistral AI brings Le Chat to iOS, Android, and enterprise [5] with web\r\nsearch and automation tools.\r\n\r\n⚡️ TRENDING SIGNALS\r\n\r\n \t*\r\nAndrew Ng presents Agentic Object Detection [6]: human-like object\r\nrecognition via text-driven reasoning.\r\n\r\n \t*\r\nGitHub upgrades Copilot with agent mode [7], multi-file edits, and an\r\nupcoming autonomous coding agent.\r\n\r\n \t*\r\nGoogle releases Imagen 3 in Gemini API [8]: high qual

In [12]:
for line in emails[0]['text'].split("TOP NEWS")[1].split("\r\n\r\n"):
    if " \t*\r" in line and re.search(r"\[\d+\]", line):
        print(line.lstrip(" \t*\r\n").strip())
        print(" ")

Mistral AI brings Le Chat to iOS, Android, and enterprise [5] with web
search and automation tools.
 
Andrew Ng presents Agentic Object Detection [6]: human-like object
recognition via text-driven reasoning.
 
GitHub upgrades Copilot with agent mode [7], multi-file edits, and an
upcoming autonomous coding agent.
 
Google releases Imagen 3 in Gemini API [8]: high quality, watermarked
image generation with improved prompt adherence.
 
OpenAI refines CoT reasoning in O3-Mini [9] models, expanding
visibility.
 
Pika Labs launches Pikadditions [10]: add any object or person to any
video with ease.
 
Google AI's AlphaGeometry2 surpasses gold medalists [11] in Olympiad
geometry, improving problem coverage and solving rates.
 
s1-32B improves test-time scaling [12] with budget forcing, boosting
math reasoning and competition scores.
 
Multi-agent system reduces AI hallucinations [13]using structured
reviews, NLP-based coordination, and new KPIs.
 


In [29]:
emails[0]['text'].split("TOP NEWS")[1].split("\r\n\r\n")

['',
 ' \t*\r\nMistral AI brings Le Chat to iOS, Android, and enterprise [5] with web\r\nsearch and automation tools.',
 '⚡️ TRENDING SIGNALS',
 ' \t*\r\nAndrew Ng presents Agentic Object Detection [6]: human-like object\r\nrecognition via text-driven reasoning.',
 ' \t*\r\nGitHub upgrades Copilot with agent mode [7], multi-file edits, and an\r\nupcoming autonomous coding agent.',
 ' \t*\r\nGoogle releases Imagen 3 in Gemini API [8]: high quality, watermarked\r\nimage generation with improved prompt adherence.',
 ' \t*\r\nOpenAI refines CoT reasoning in O3-Mini [9] models, expanding\r\nvisibility.',
 ' \t*\r\nPika Labs launches Pikadditions [10]: add any object or person to any\r\nvideo with ease.',
 '💻TOP PAPERS',
 " \t*\r\nGoogle AI's AlphaGeometry2 surpasses gold medalists [11] in Olympiad\r\ngeometry, improving problem coverage and solving rates.",
 ' \t*\r\ns1-32B improves test-time scaling [12] with budget forcing, boosting\r\nmath reasoning and competition scores.',
 ' \t*\r\nMu

In [13]:
SECTIONS = [
    "TOP NEWS",
    "TRENDING SIGNALS",
    "TOP PAPERS",
    "PYTHON TIP",
]
txt = emails[0]['text']
sections = []
for sec in SECTIONS:
    _tmp = txt.split(sec)
    sections.append(_tmp[0])
    txt = _tmp[1]

In [15]:
for pp in sections[1:]:
    print(pp)



 	*
Mistral AI brings Le Chat to iOS, Android, and enterprise [5] with web
search and automation tools.

⚡️ 


 	*
Andrew Ng presents Agentic Object Detection [6]: human-like object
recognition via text-driven reasoning.

 	*
GitHub upgrades Copilot with agent mode [7], multi-file edits, and an
upcoming autonomous coding agent.

 	*
Google releases Imagen 3 in Gemini API [8]: high quality, watermarked
image generation with improved prompt adherence.

 	*
OpenAI refines CoT reasoning in O3-Mini [9] models, expanding
visibility.

 	*
Pika Labs launches Pikadditions [10]: add any object or person to any
video with ease.

💻


 	*
Google AI's AlphaGeometry2 surpasses gold medalists [11] in Olympiad
geometry, improving problem coverage and solving rates.

 	*
s1-32B improves test-time scaling [12] with budget forcing, boosting
math reasoning and competition scores.

 	*
Multi-agent system reduces AI hallucinations [13]using structured
reviews, NLP-based coordination, and new KPIs.

🧠 


In [19]:
print(emails[0]['text'].split("TOP NEWS")[2].split("WHAT'S NEW")[1].split("TRENDING SIGNALS")[0].split("TRY NOW")[0])



French AI lab, Mistral AI updates its chatbot LE CHAT and makes it
available on iOS, Android, and enterprise infrastructure. It
introduces FLASH ANSWERS, A BUILT-IN CODE INTERPRETER, and REAL-TIME
SEARCH, competing with ChatGPT, Claude, and DeepSeek.

CORE FUNCTIONALITIES

Le Chat provides tools for both general use and technical
applications, offering:

 	* OCR ENGINE: Processes PDFs, spreadsheets, and complex or
low-quality files.

 	* CODE INTERPRETER: Runs scripts, handles data analysis, and creates
visualizations.

 	* IMAGE GENERATION: Uses BLACK FOREST LABS’ FLUX ULTRA to generate
photorealistic visuals.

 	* REAL-TIME INFORMATION RETRIEVAL: Combines static pre-trained
knowledge with updated data from web searches and news outlets.

 	* Its MULTI-STEP AGENT FRAMEWORK lets you AUTOMATE WORKFLOWS by
integrating with messaging systems and databases.

PERFORMANCE

Le Chat runs on Mistral’s latest models and introduces FLASH
ANSWERS, a feature enabling RESPONSE SPEEDS OF UP TO 1,00

# TLDR

In [14]:
emails = gmail.fetch_emails(sender='TLDR AI <dan@tldrnewsletter.com>', after='2024-12-01', before='2024-12-07')
len(emails)

5

In [15]:
print(emails[0])

{'sender': 'TLDR AI <dan@tldrnewsletter.com>', 'subject': "OpenAI o1 System Card 📇, Perplexity Publisher Program 🔍, DeepMind's Genie 2 🧞", 'date_utc': 'Fri, 6 Dec 2024 14:16:24 +0000', 'id': '1939c5434a9f7fd8', 'text': "This report outlines the safety work carried out prior to releasing\r\nOpenAI o1 and o1-mini, including external red teaming and frontier\r\nrisk\r\nevaluation\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\r\n\r\n\r\n Sign Up [1] |Advertise [2]|View Online [3] \r\n\r\n\t\tTLDR \r\n\r\n\t\tTOGETHER

In [16]:
news_sources = tldr_parser(emails[0])

In [7]:
news_sources

[{'title': 'OPENAI O1 SYSTEM CARD',
  'url': 'https://openai.com/index/openai-o1-system-card/',
  'source_of_the_news': 'TLDR AI <dan@tldrnewsletter.com>',
  'news_summary': "This report outlines the safety work carried out prior to releasing OpenAI o1 and o1-mini, including external red teaming and frontier risk evaluations according to OpenAI's Preparedness Framework.",
  'date_source': 'Fri, 6 Dec 2024 14:16:24 +0000',
  'new_provider': 'openai.com'},
 {'title': "PERPLEXITY EXPANDING IT'S PUBLISHER'S PROGRAM",
  'url': 'https://www.perplexity.ai/hub/blog/perplexity-expands-publisher-program-with-15-new-media-partners',
  'source_of_the_news': 'TLDR AI <dan@tldrnewsletter.com>',
  'news_summary': "Perplexity has added over a dozen international news organizations to its Publishers' Program, offering tools, revenue sharing, and support to strengthen collaboration with global media.",
  'date_source': 'Fri, 6 Dec 2024 14:16:24 +0000',
  'new_provider': 'www.perplexity.ai'},
 {'title': 

In [8]:
links = emails[0]['text'].split("\r\n\r\n")[-1].split('\r\n')
links

['Links:',
 '------',
 '[1] https://tldr.tech/ai?utm_source=tldrai',
 '[2] https://advertise.tldr.tech/?utm_source=tldrai&utm_medium=newsletter&utm_campaign=advertisetopnav',
 '[3] https://a.tldrnewsletter.com/web-version?ep=1&lc=57aff87c-ae02-11ef-9074-4b7e3c69f0c6&p=68d03084-b3c7-11ef-928d-09d27092d252&pt=campaign&t=1733494584&s=a94ebb372672f7844f8d11fcad3029c68791b13faf831d296ca3c8f639256320',
 '[4] https://writer.com/product/ai-studio/?utm_source=tldr&utm_medium=newsletter&utm_campaign=ai_studio',
 '[5] https://links.tldrnewsletter.com/xS4pwo',
 '[6] https://links.tldrnewsletter.com/UUkpwY',
 '[7] https://techcrunch.com/2024/12/04/deepminds-genie-2-can-generate-interactive-worlds-that-look-like-video-games/?utm_source=tldrai',
 '[8] https://arxiv.org/abs/2412.03555?utm_source=tldrai',
 '[9] https://arxiv.org/abs/2412.02044v1?utm_source=tldrai',
 '[10] https://arxiv.org/abs/2412.03248v1?utm_source=tldrai',
 '[11] https://page.camunda.com/wp-rethinking-change-management?utm_medium=pa

In [14]:
"tldr" in links[6]

True

In [9]:
links[10].split('] ')

['[9', 'https://arxiv.org/abs/2412.02044v1?utm_source=tldrai']

In [10]:
type(news_sources[0]['url']), news_sources[0]['url']

(str, 'https://openai.com/index/openai-o1-system-card/')